In [1]:
# do not touch this cell 
%load_ext jupyter_record


# Set-up

Make sure that you have completed the set-up instuctions in Assignment 0. 

Every time you create a Jupyter cell in this assignment, it should be annotated with '%%git_commit' at the top. Please do not touch this line of code. If you do not see this line, make sure that you have moved the 'custom.js' file in the right location!


# Data Preprocessing

In this section, you should load your raw training dataset, and create your features for the model. You can find the data here: [shorturl.at/owMPS](https://shorturl.at/owMPS)

**IMPORTANT**: Download your data and place it in the "data" folder. You may run into errors if you place it anywhere else. You can use pandas to import your data from the csv files.

The data files are as following:

- training.csv: Seen/training data. Contains raw features and prediction class ('DepDel15').
- testing_X.csv: Unseen/testing data. Contains only raw features, and no prediction class. 
- readme.html: Information about the features. 



In [2]:
%%git_commit

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/training.csv')
print(df.head())

   Unnamed: 0  FlightDate            Airline Origin Dest  Cancelled  Diverted  \
0           0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False   
1           1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False   
2           2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False   
3           3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False   
4           4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False   

   CRSDepTime  CRSElapsedTime  Distance  ...  DestAirportID  DestAirportSeqID  \
0        1202            62.0     145.0  ...          10397           1039707   
1        1202            62.0     145.0  ...          10397           1039707   
2        1202            62.0     145.0  ...          10397           1039707   
3        1202            62.0     145.0  ...          10397           1039707   
4        1400            60.0     145.0  ...          10397           1039707   

   DestCityMarketID  DestC

# Model Prediction

In this section, you should import relevant modeling/scikit-learn libraries and create a prediction model from your features. You should carefuly evaluate how your model performs based on the data you already have.

**Hint**: We are using a real world dataset with low correlation between the raw features and prediction class. You may have to carefully think about how to hand-engineer features from the raw features to capture correlation using a machine learning model.


**Hint 2**: When the instructors ran this assignment, we achieved an accuracy of 0.74 and AUC score of 0.54 on held out data. We do not expect high accuracy for this task, but your goal is to create a prediction model that is better than the simplest baseline - i.e. a model that predictions the majority class for all samples.


In [9]:
%%git_commit
df.dropna(inplace = True)
result = df.select_dtypes(include='number')

print(result.head())
cr = df.corr()
cr = cr[abs(cr["DepDel15"]) > .02]
print(cr)

X = result[['CRSDepTime','CRSArrTime']].to_numpy()
Y = result['DepDel15'].to_numpy()

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
print(df.shape)

   Unnamed: 0  CRSDepTime  CRSElapsedTime  Distance  Year  Quarter  Month  \
0           0        1202            62.0     145.0  2018        1      1   
1           1        1202            62.0     145.0  2018        1      1   
2           2        1202            62.0     145.0  2018        1      1   
3           3        1202            62.0     145.0  2018        1      1   
4           4        1400            60.0     145.0  2018        1      1   

   DayofMonth  DayOfWeek  DOT_ID_Marketing_Airline  ...  OriginCityMarketID  \
0          23          2                     19790  ...               30146   
1          24          3                     19790  ...               30146   
2          25          4                     19790  ...               30146   
3          26          5                     19790  ...               30146   
4          27          6                     19790  ...               30146   

   OriginStateFips  OriginWac  DestAirportID  DestAirportSeqID

/var/folders/0_/qzbnbnbs67sfl9l1gk_xy9qr0000gn/T/ipykernel_3272/4046197561.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cr = df.corr()


            Unnamed: 0  Cancelled  Diverted  CRSDepTime  CRSElapsedTime  \
Unnamed: 0    1.000000   0.002335  0.005062    0.002603       -0.000369   
CRSDepTime    0.002603   0.005323  0.000099    1.000000       -0.018113   
DepDel15      0.025902   0.019524  0.017706    0.172097        0.013892   
CRSArrTime   -0.010884   0.003573  0.004930    0.684092        0.023671   

            Distance  Year   Quarter     Month  DayofMonth  ...  \
Unnamed: 0  0.015777   NaN -0.109568 -0.003606   -0.001773  ...   
CRSDepTime -0.010256   NaN -0.004631 -0.004569   -0.000125  ...   
DepDel15    0.012557   NaN -0.019862 -0.018527   -0.004709  ...   
CRSArrTime  0.018664   NaN  0.005901  0.005352   -0.001685  ...   

            OriginCityMarketID  OriginStateFips  OriginWac  DestAirportID  \
Unnamed: 0           -0.004169         0.001619   0.009182       0.005452   
CRSDepTime           -0.047597        -0.024678  -0.007698       0.021534   
DepDel15             -0.017602        -0.004494  -0.00991

In [ ]:
%%git_commit
print(X)

In [4]:
%%git_commit
from sklearn.linear_model import LogisticRegression

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33)

print(len(X_train))
model = LogisticRegression()
model.fit(X_train,Y_train)

Y_hat = model.predict(X_train)


3753963


In [6]:
%%git_commit
# (sum(Y_test) - sum(Y_hat))/len(Y_test[Y_test == 1])

Y_hat


array([0., 0., 0., ..., 0., 0., 0.])

# Final Evaluation

In this section, you should use your prediction model on held out data. For each sample in 'testing_X.csv', generate a binary prediction on whether the flight will be delayed by 15min or more.

**Important**: Create a csv file named that is a list of your predictions. It should be named './data/[CNET_ID]_final_assignment.csv' file, and contain only one row and columns equal to the sample size. 

**IMPORTANT**: Make sure that you create the new file in the "data" folder. You may run into errors if you place it anywhere else. 

Upload the '.csv' file here: [shorturl.at/aox05](https://shorturl.at/aox05)

This is your submission for the project! A part of your assessment will be on performance on this unseen dataset, and whether you can do better than majority class prediction. 

Note that you do not have true classification labels on this dataset.


In [ ]:
%%git_commit
df2 = pd.read_csv('data/testing_X.csv')
df2.head()


In [ ]:
%%git_commit
